In [14]:
# Read the csv dataset
import py_entitymatching as em
A = em.read_csv_metadata('../datasets/yelp_data.csv')
A['ID'] = range(0, len(A))
em.set_key(A, 'ID')

B = em.read_csv_metadata('../datasets/inspection_data.csv')
B['ID'] = range(0, len(B))
em.set_key(B, 'ID')



Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


True

In [15]:
A.head()

,name,address,zipcode,cuisine,ID
0,barn joo 35,"34 w 35th st, ,",10001,korean gastropubs tapas/small plates,0
1,friedman's,"132 w 31st st, ,",10001,american (new) gluten-free breakfast & brunch,1
2,juniper,"237 w 35th st,",10001,cocktail bars american (new),2
3,haymaker bar and kitchen,"252 w 29th st, ,",10001,gastropubs beer bar,3
4,izakaya mew,"53 w 35th st, basement,",10001,sushi bars izakaya cocktail bars,4


## Down-sample

In [16]:
sample_A, sample_B = em.down_sample(A, B, size=18000, y_param=2)
# Save the datasets because it changes every time we perform down-sampling.
em.to_csv_metadata(sample_A, '../datasets/sample_A_generated.csv')
em.to_csv_metadata(sample_B, '../datasets/sample_B_generated.csv')

0%                          100%
[##                            ] | ETA: 00:00:37 | ETA: 00:00:33

KeyboardInterrupt: 

## Blocking

In [ ]:
# Perform blocking
ab = em.AttrEquivalenceBlocker()
C = ab.block_tables(sample_A, sample_B, 'zipcode', 'zipcode', l_output_attrs=['name', 'address', 'zipcode', 'cuisine'], r_output_attrs=['name', 'address', 'zipcode', 'cuisine'])
ob = em.OverlapBlocker()
D = ob.block_candset(C, 'address', 'address', overlap_size=4)

In [ ]:
# Describe the candidate set
print(len(D))
D.head(2)

In [ ]:
# Sample the candidate set.
S = em.sample_table(D, sample_size=1000) # C is the candidate set to be sampled from.
# Write the sample set to a file for further processing.
em.to_csv_metadata(S, '../datasets/candidate_sample_generated.csv')

## Feature extraction

In [41]:
# Loading the saved files
import py_entitymatching as em
sample_A = em.read_csv_metadata('../datasets/sample_A.csv')
sample_B = em.read_csv_metadata('../datasets/sample_B.csv')
S = em.read_csv_metadata('../datasets/candidate_sample.csv')
G = em.read_csv_metadata('../datasets/candidate_sample_labeled.csv')

# Adding metadata to the files
em.set_key(sample_A, 'ID')
em.set_key(sample_B, 'ID')

em.set_key(S, '_id')
em.set_fk_ltable(S, 'ltable_ID')
em.set_fk_rtable(S, 'rtable_ID')
em.set_ltable(S, sample_A)
em.set_rtable(S, sample_B)

em.set_key(G, '_id')
em.set_fk_ltable(G, 'ltable_ID')
em.set_fk_rtable(G, 'rtable_ID')
em.set_ltable(G, sample_A)
em.set_rtable(G, sample_B)
IJ = em.split_train_test(G, train_proportion=0.5, random_state=0)
I = IJ['train']
J = IJ['test']

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [42]:
# Extracting features
match_f = em.get_features_for_matching(sample_A, sample_B)
H = em.extract_feature_vecs(I, feature_table=match_f,attrs_after=['label'])
H.head()

0%                          100%
[##############################] | ETA: 00:00:05 | ETA: 00:00:04 | ETA: 00:00:04 | ETA: 00:00:04 | ETA: 00:00:04 | ETA: 00:00:04 | ETA: 00:00:04 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:05


,_id,ltable_ID,rtable_ID,name_name_jac_qgm_3_qgm_3,name_name_cos_dlm_dc0_dlm_dc0,name_name_jac_dlm_dc0_dlm_dc0,name_name_mel,name_name_lev_dist,name_name_lev_sim,name_name_nmw,...,cuisine_cuisine_mel,cuisine_cuisine_lev_dist,cuisine_cuisine_lev_sim,cuisine_cuisine_nmw,cuisine_cuisine_sw,ID_ID_exm,ID_ID_anm,ID_ID_lev_dist,ID_ID_lev_sim,label
631,1003068,5272,20785,0.000000,0.000000,0.000000,0.549242,9.0,0.181818,-2.0,...,0.398148,14.0,0.125000,-5.0,2.0,0,0.253644,4,0.2,0
185,469420,2029,20213,0.033333,0.000000,0.000000,0.535000,11.0,0.266667,1.0,...,0.531490,27.0,0.129032,1.0,4.0,0,0.100381,2,0.6,0
732,1123677,6975,23836,0.285714,0.666667,0.500000,0.867557,8.0,0.600000,9.0,...,1.000000,0.0,1.000000,7.0,7.0,0,0.292625,5,0.0,1
404,830665,3643,12177,0.666667,0.500000,0.333333,0.981818,1.0,0.909091,9.0,...,0.495726,11.0,0.153846,-5.0,2.0,0,0.299171,5,0.0,1
194,476336,2057,22820,0.000000,0.000000,0.000000,0.390476,19.0,0.095238,-4.0,...,0.420635,16.0,0.111111,-9.0,1.0,0,0.090140,4,0.2,0


## Matching

In [43]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
nb = em.NBMatcher(name='NaiveBayes')
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')

In [44]:
# Report Precision
result = em.select_matcher([dt, nb, rf, svm, ln, lg], table=H, 
        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
        k=5,
        target_attr='label', metric='precision', random_state=0)
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x118030f28>,5,0.725000,0.757576,0.787879,0.804878,0.757576,0.766582
1,NaiveBayes,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x118030fd0>,5,0.780000,0.750000,0.744186,0.833333,0.729730,0.767450
2,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x1177400b8>,5,0.808511,0.800000,0.789474,0.833333,0.725000,0.791264
3,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x118030cc0>,5,0.794118,0.857143,0.800000,0.814815,0.730769,0.799369
4,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x117740cc0>,5,0.754717,0.744186,0.767442,0.844444,0.690476,0.760253
5,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x117740860>,5,0.770833,0.837838,0.756098,0.818182,0.684211,0.773432


In [45]:
# Report Recall
# Select the best ML matcher using CV
result = em.select_matcher([dt, nb, rf, svm, ln, lg], table=H, 
        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
        k=5,
        target_attr='label', metric='recall', random_state=0)
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x118030f28>,5,0.725,0.714286,0.702703,0.804878,0.757576,0.740888
1,NaiveBayes,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x118030fd0>,5,0.975,0.857143,0.864865,0.853659,0.818182,0.873770
2,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x1177400b8>,5,0.950,0.800000,0.810811,0.731707,0.878788,0.834261
3,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x118030cc0>,5,0.675,0.685714,0.432432,0.536585,0.575758,0.581098
4,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x117740cc0>,5,1.000,0.914286,0.891892,0.926829,0.878788,0.922359
5,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x117740860>,5,0.925,0.885714,0.837838,0.878049,0.787879,0.862896


In [46]:
# Report F1
# Select the best ML matcher using CV
result = em.select_matcher([dt, nb, rf, svm, ln, lg], table=H, 
        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
        k=5,
        target_attr='label', metric='f1', random_state=0)
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x118030f28>,5,0.725000,0.735294,0.742857,0.804878,0.757576,0.753121
1,NaiveBayes,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x118030fd0>,5,0.866667,0.800000,0.800000,0.843373,0.771429,0.816294
2,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x1177400b8>,5,0.873563,0.800000,0.800000,0.779221,0.794521,0.809461
3,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x118030cc0>,5,0.729730,0.761905,0.561404,0.647059,0.644068,0.668833
4,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x117740cc0>,5,0.860215,0.820513,0.825000,0.883721,0.773333,0.832556
5,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x117740860>,5,0.840909,0.861111,0.794872,0.847059,0.732394,0.815269


# Debug Matcher

# Find Matcher after debugging

In [52]:
# Train using feature vectors from I
matchers = [dt, nb, rf, svm, ln, lg];
names = ['Decision Tree', 'Naive Bayes', 'Random Forest', 'SVM', 'Linear Regression', 'Logistic Regression']
for i in range(0,len(matchers)):
    matcher = matchers[i]
    name = names[i]
    matcher.fit(table=H, 
           exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
           target_attr='label')

    # Convert J into a set of feature vectors using F
    L = em.extract_feature_vecs(J, feature_table=match_f,attrs_after=['label'])

    # Predict on L 
    predictions = matcher.predict(table=L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
                  append=True, target_attr='predicted', inplace=False)
    # Evaluate the predictions
    eval_result = em.eval_matches(predictions, 'label', 'predicted')
    print(name)
    em.print_eval_summary(eval_result)

0%                          100%
[##############################] | ETA: 00:00:04 | ETA: 00:00:04 | ETA: 00:00:04 | ETA: 00:00:04 | ETA: 00:00:04 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:04
0%                          100%
[#                             ] | ETA: 00:00:03

Decision Tree
Precision : 80.81% (139/172)
Recall : 69.85% (139/199)
F1 : 74.93%
False positives : 33 (out of 172 positive predictions)
False negatives : 60 (out of 328 negative predictions)


[##############################] | ETA: 00:00:04 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:04
0%                          100%
[#                             ] | ETA: 00:00:03

Naive Bayes
Precision : 81.73% (170/208)
Recall : 85.43% (170/199)
F1 : 83.54%
False positives : 38 (out of 208 positive predictions)
False negatives : 29 (out of 292 negative predictions)


[##############################] | ETA: 00:00:04 | ETA: 00:00:03 | ETA: 00:00:04 | ETA: 00:00:04 | ETA: 00:00:04 | ETA: 00:00:04 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:04
0%                          100%
[#                             ] | ETA: 00:00:03

...

[##############################] | ETA: 00:00:04 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:05
0%                          100%
[#                             ] | ETA: 00:00:04

Linear Regression
Precision : 80.0% (180/225)
Recall : 90.45% (180/199)
F1 : 84.91%
False positives : 45 (out of 225 positive predictions)
False negatives : 19 (out of 275 negative predictions)


[##############################] | ETA: 00:00:04 | ETA: 00:00:05 | ETA: 00:00:05 | ETA: 00:00:05 | ETA: 00:00:05 | ETA: 00:00:04 | ETA: 00:00:04 | ETA: 00:00:04 | ETA: 00:00:04 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00

Logistic Regression
Precision : 82.86% (174/210)
Recall : 87.44% (174/199)
F1 : 85.09%
False positives : 36 (out of 210 positive predictions)
False negatives : 25 (out of 290 negative predictions)



Total time elapsed: 00:00:05
